In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

def processar_planilhas():
    recebimento = pd.read_excel('recebimento_ajustado.xlsx')
    expedicao = pd.read_excel('expedicao.xlsx')
    lista_produtos = pd.read_excel('listadeprodutos.xlsx')
    matriz_distancias = pd.read_excel('matriz_distancias.xlsx', index_col=0)

    expedicao = expedicao.rename(columns={
        'Nº da ordem de transporte': 'ordem',
        'Material': 'material',
        'Qtd.teórica origem': 'quantidade',
        'Posição dep.origem': 'origem',
        'Posição de destino enclausuramentos': 'destino',
        'Data de criação': 'data',
        'Hora da criação': 'hora'
    })

    recebimento = recebimento.rename(columns={
        'Nº da ordem de transporte': 'ordem',
        'tipo de produto': 'material',
        'Qtd.teórica origem': 'quantidade',
        'posição de origem ': 'origem',
        'Posição dep.destino': 'destino',
        'Data de criação': 'data',
        'Hora da criação': 'hora'
    })

    def padronizar_localizacao(loc):
      if pd.isna(loc):
          return loc
      loc = str(loc)

      if loc.isdigit():
          return f"e{loc.zfill(2)}"

      elif re.match(r'^A[VX][A-Za-z0-9]+$', loc):
          return loc.upper().strip()

      elif loc.startswith(('Esteira', 'E')):
          return loc

      else:
          return loc.upper().strip()

    expedicao['destino'] = expedicao['destino'].apply(padronizar_localizacao)
    recebimento['origem'] = recebimento['origem'].apply(padronizar_localizacao)
    expedicao['operacao'] = 'expedicao'
    recebimento['operacao'] = 'recebimento'

    df_unificado = pd.concat([recebimento, expedicao], ignore_index=True)

    df_unificado['data_hora'] = pd.to_datetime(
        df_unificado['data'].astype(str) + ' ' + df_unificado['hora'].astype(str))

    lista_produtos = lista_produtos.rename(columns={
        'PNC': 'material',
        'Base Bloco': 'base'
    })
    df_unificado = pd.merge(df_unificado, lista_produtos[['material', 'base']],
                           on='material', how='left')

    def calcular_distancia(row):
        origem = row['origem']
        destino = row['destino']

        if origem in matriz_distancias.columns and destino in matriz_distancias.index:
            return matriz_distancias.loc[destino, origem]
        return np.nan

    df_unificado['distancia'] = df_unificado.apply(calcular_distancia, axis=1)

    df_unificado = df_unificado.sort_values('data_hora').reset_index(drop=True)
    colunas_finais = [
        'operacao', 'ordem', 'material', 'base', 'quantidade',
        'origem', 'destino', 'data_hora', 'distancia'
    ]
    df_unificado = df_unificado[colunas_finais]
    df_unificado.to_excel('ordens_unificadas.xlsx', index=False)
    print("Planilha criada")

    return df_unificado

df_final = processar_planilhas()

Planilha unificada criada com sucesso!


In [1]:
import pandas as pd
import numpy as np

matriz_distancias = pd.read_excel('matriz_distancias.xlsx')
sample = pd.read_excel('planilhaDiminuida.xlsx')

In [6]:
sample = sample.dropna()
sample.head()

,operacao,ordem,material,base,quantidade,origem,destino,data_hora,distancia
0,expedicao,2481001,900941452,6,90,AVF0220,e13,2019-11-01 00:01:35,140.2
1,expedicao,2481002,900941452,6,72,AVF0220,e13,2019-11-01 00:01:49,140.2
2,expedicao,2481006,900941452,6,54,AVF0220,e13,2019-11-01 00:06:24,140.2
3,expedicao,2481007,900941452,6,36,AVF0220,e13,2019-11-01 00:06:39,140.2
9,expedicao,2481015,900941452,6,18,AVF0220,e13,2019-11-01 00:28:29,140.2


In [18]:
lista_sample_origem_valores_unicos = sample['origem'].unique()
lista_sample_origem_valores_unicos
lista_sample_destino_valores_unicos = sample['destino'].unique()
lista_sample_destino_valores_unicos
#lista_sample_valores_unicos = lista_sample_origem_valores_unicos + lista_sample_destino_valores_unicos
lista_sample_destino_valores_unicos = list(lista_sample_destino_valores_unicos)
lista_sample_origem_valores_unicos = list(lista_sample_origem_valores_unicos)
lista_sample_valores_unicos = lista_sample_destino_valores_unicos + lista_sample_origem_valores_unicos
lista_sample_valores_unicos

['e13',
 'AVG0105',
 'AVA0121',
 'AVC0202',
 'AVD0116',
 'AVC0218',
 'AVD0107',
 'AVB0215',
 'AVB0207',
 'AVB0202',
 'AVC0113',
 'AVH0105',
 'AVF0108',
 'AVG0202',
 'AVC0115',
 'AVH0218',
 'AVB0224',
 'AVC0208',
 'AVH0103',
 'AVH0216',
 'AVD0109',
 'AVB0220',
 'AVG0218',
 'AVB0107',
 'AVD0202',
 'AVC0222',
 'AVF0107',
 'AVA0209',
 'AVB0208',
 'AVG0102',
 'AVE0119',
 'AVB0128',
 'AVA0219',
 'AVA0231',
 'AVB0110',
 'AVF0130',
 'AVB0106',
 'AVG0216',
 'AVG0201',
 'AVG0108',
 'AVD0209',
 'AVF0120',
 'AVB0105',
 'AVC0116',
 'AVG0123',
 'AVB0118',
 'AVF0220',
 'Esteira02',
 'Esteira03']

In [28]:
matriz_distancias = pd.read_excel('matriz_distancias.xlsx', index_col=0)

submatriz = matriz_distancias.loc[lista_sample_valores_unicos, lista_sample_valores_unicos]
submatriz
submatriz.to_excel('matriz_filtrada.xlsx')